Test for one course  price , syllabus , about course , course faculty , language

In [2]:
import os
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options   # 👈 Add this
from webdriver_manager.chrome import ChromeDriverManager

# -------------------- DRIVER SETUP --------------------
def get_driver(headless=False):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

# === Input: One Course URL ===
course_link = "https://www.eduonix.com/business-data-analytics-bootcamp-master-ai-powered-insights"

# === Setup Headless Chrome ===
chrome_options = Options()
chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=chrome_options)

# === Utility: Check if text is a question ===
def is_question(text):
    question_starters = ['why', 'how', 'what', 'when', 'can', 'do', 'does', 'is', 'are', 'which']
    text = text.strip().lower()
    return text.endswith('?') or any(text.startswith(q) for q in question_starters)

# === Load and Parse Page ===
driver.get(course_link)
time.sleep(3)
soup = BeautifulSoup(driver.page_source, 'html.parser')

# === Extract Course Name ===
course_name_tag = soup.find('h1', class_='productTopHeading')
course_name = course_name_tag.get_text(strip=True) if course_name_tag else "Unknown Course Name"

# === Extract Language ===
language = "Unknown"
lang_spans = soup.find_all('span', class_='divInfo')
for span in lang_spans:
    if 'fa-globe' in str(span):
        language = span.get_text(strip=True)
        break

# === Extract Instructor ===
faculty = "Unknown"
instructor_block = soup.find('b', string="Instructor:")
if instructor_block and instructor_block.next_sibling:
    faculty_tag = instructor_block.find_next('a')
    if faculty_tag:
        faculty = faculty_tag.get_text(strip=True)

# === Extract Price ===
price_tag = soup.find('span', id='mnumber')
price = price_tag.get_text(strip=True) if price_tag else "Unknown Price"

# === Extract About Course ===
about_course = "Not Available"
about_div = soup.find('div', class_='viewMoreDescription')
if about_div:
    about_course = '\n'.join(p.get_text(strip=True) for p in about_div.find_all('p') if p.get_text(strip=True))

# === Extract Syllabus ===
# === Extract All Course Titles ===
course_titles = []
for btn in soup.find_all('button', class_='edHead'):
    title_tag = btn.find('span', class_='edCardTitle')
    if title_tag:
        title = title_tag.get_text(strip=True)
        if title:
            course_titles.append(title)

# === Extract All Syllabus Content ===
syllabus_blocks = soup.find_all(['div', 'span', 'h4'])

syllabus_content = ""
current_course_index = -1

for block in syllabus_blocks:
    if block.name == 'span' and 'edCardTitle' in block.get('class', []):
        current_course_index += 1
        syllabus_content += f"\n🎓 {course_titles[current_course_index]}\n"

    elif block.name == 'h4' and 'curriculumName' in block.get('class', []):
        module_title_elem = block
        module_num_elem = block.find_previous('div', class_='moduleHeading')
        module_title = module_title_elem.get_text(strip=True)
        module_num = module_num_elem.get_text(strip=True) if module_num_elem else ''
        full_module_title = f"{module_num} {module_title}".strip()
        syllabus_content += f"\n🔸 {full_module_title}\n"

    elif block.name == 'span' and 'secInfo' in block.get('class', []):
        b_tag = block.find('b')
        if b_tag:
            title = b_tag.get_text(strip=True)
            if 'frequently asked' in title.lower() or 'faq' in title.lower() or is_question(title):
                continue
            syllabus_content += f"\n📘 {title.strip()}\n"

    elif block.name == 'div' and 'syllbusMain' in block.get('class', []):
        desc_div = block.find('div', class_='syllbusDesc')
        if desc_div:
            for btn in desc_div.find_all('button'):
                btn.decompose()
            lesson_text = desc_div.get_text(strip=True)
            if lesson_text:
                syllabus_content += f"- {lesson_text.strip()}\n"


driver.quit()

# === Final Output ===
if not syllabus_content.strip():
    syllabus_content = "No Syllabus Found"

# === Print Results ===
print(f"📗 Course Name: {course_name}")
print(f"🌐 Language: {language}")
print(f"👨‍🏫 Course Faculty: {faculty}")
print(f"💰 Price: {price}")
print(f"\n📝 About Course:\n{about_course}\n")
print(syllabus_content.strip())


📗 Course Name: Business Data Analytics Bootcamp: Master AI Powered Insights
🌐 Language: English (India)
👨‍🏫 Course Faculty: Metla Sudha Sekhar
💰 Price: ₹995.00

📝 About Course:
Unlock the power of data and transform your decision making with ourBusiness Data Analytics Bootcamp. Designed for professionals and aspiring analysts, this comprehensive course will equip you with the essential skills to extract valuable insights from data using industry leading tools like Excel, SQL, Power BI, Python, Pandas, Matplotlib, and ChatGPT AI.
Applying these tools to automate workflows, optimize performance, enhancedecision making, streamline operations, uncover trends, actionable insights, boost productivity, gain competitive advantage, transform data, business growth.
Throughout this hands on journey, you will learn to clean, analyze, and visualize data, build interactive dashboards, and leverageAI powered analyticsto make smarter business decisions. Master SQL for efficient data extraction, use Po

EXTRACTED  FOR ALL COURSES WEB SCRAPPER  # IMP

In [2]:
import os
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options   # 👈 Add this
from webdriver_manager.chrome import ChromeDriverManager

# -------------------- DRIVER SETUP --------------------
def get_driver(headless=False):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# === Input Excel file ===
input_excel_path = r"C:\Users\taslim.siddiqui\Downloads\Eduonix course link _NEW.xlsx"
df_links = pd.read_excel(input_excel_path)
course_links = df_links['Course Link'].dropna().tolist()

# === Setup Chrome ===
chrome_options = Options()
chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=chrome_options)

# === Utility ===
def is_question(text):
    question_starters = ['why', 'how', 'what', 'when', 'can', 'do', 'does', 'is', 'are', 'which']
    text = text.strip().lower()
    return text.endswith('?') or any(text.startswith(q) for q in question_starters)

# === Scrape each course ===
all_data = []

for i, link in enumerate(course_links):
    print(f"Scraping ({i+1}/{len(course_links)}): {link}")
    try:
        driver.get(link)
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # --- Course Name ---
        course_name_tag = soup.find('h1', class_='productTopHeading')
        course_name = course_name_tag.get_text(strip=True) if course_name_tag else "Unknown Course Name"

        # --- Language ---
        language = "Not found"
        lang_spans = soup.find_all('span', class_='divInfo')
        for span in lang_spans:
            if 'fa-globe' in str(span):
                language = span.get_text(strip=True)
                break

        # --- Course Faculty ---
        faculty = "Not found"
        instructor_tag = soup.find('a', href=lambda x: x and '/i/' in x)
        if instructor_tag:
            faculty = instructor_tag.get_text(strip=True)

        # --- Price ---
        price = "Not found"
        price_tag = soup.find('span', id='mnumber')
        if price_tag:
            price_text = price_tag.get_text(strip=True)
            price = price_text.replace('\xa0', ' ').replace('₹', ' ').strip()
        else:
            strike_tag = soup.find('strike', class_='original_price')
            if strike_tag:
                price_text = strike_tag.get_text(strip=True)
                price = price_text.replace('\xa0', ' ').replace('₹', ' ').strip()
            else:
                cut_price_tag = soup.find('div', class_='cutPrice')
                if cut_price_tag:
                    price_text = cut_price_tag.get_text(strip=True)
                    price = price_text.replace('\xa0', ' ').replace('₹', '').strip()

        # --- Certificate ---
        certificate = "No"
        certificate_div = soup.find('div', class_='courseCertificate')
        if certificate_div:
            certificate = "Yes"

        # --- About Course ---
        about_course = "Not Available"
        about_div = soup.find('div', class_='viewMoreDescription')
        if about_div:
            about_course = '\n'.join(
                p.get_text(strip=True) for p in about_div.find_all('p') if p.get_text(strip=True)
            )

        # --- Course Syllabus ---
        course_titles = []
        for btn in soup.find_all('button', class_='edHead'):
            title_tag = btn.find('span', class_='edCardTitle')
            if title_tag:
                title = title_tag.get_text(strip=True)
                if title:
                    course_titles.append(title)

        syllabus_blocks = soup.find_all(['div', 'span', 'h4'])

        syllabus_content = ""
        current_course_index = -1

        for block in syllabus_blocks:
            if block.name == 'span' and 'edCardTitle' in block.get('class', []):
                current_course_index += 1
                if current_course_index < len(course_titles):
                    syllabus_content += f"\n🎓 {course_titles[current_course_index]}\n"

            elif block.name == 'h4' and 'curriculumName' in block.get('class', []):
                module_title_elem = block
                module_num_elem = block.find_previous('div', class_='moduleHeading')
                module_title = module_title_elem.get_text(strip=True)
                module_num = module_num_elem.get_text(strip=True) if module_num_elem else ''
                full_module_title = f"{module_num} {module_title}".strip()
                syllabus_content += f"\n🔸 {full_module_title}\n"

            elif block.name == 'span' and 'secInfo' in block.get('class', []):
                b_tag = block.find('b')
                if b_tag:
                    title = b_tag.get_text(strip=True)
                    if 'frequently asked' in title.lower() or 'faq' in title.lower() or is_question(title):
                        continue
                    syllabus_content += f"\n📘 {title.strip()}\n"

            elif block.name == 'div' and 'syllbusMain' in block.get('class', []):
                desc_div = block.find('div', class_='syllbusDesc')
                if desc_div:
                    for btn in desc_div.find_all('button'):
                        btn.decompose()
                    lesson_text = desc_div.get_text(strip=True)
                    if lesson_text:
                        syllabus_content += f"- {lesson_text.strip()}\n"

        # === Save course data ===
        course_data = {
            'Course Name': course_name,
            'Course Link': link,
            'Language': language,
            'Course Faculty': faculty,
            'Price': price,
            'Certificate': certificate,
            'About Course': about_course,
            'Course Syllabus': syllabus_content.strip()
        }

    except Exception as e:
        print(f"❌ Error scraping {link}: {e}")
        course_data = {
            'Course Name': '',
            'Course Link': link,
            'Language': '',
            'Course Faculty': '',
            'Price': '',
            'Certificate': '',
            'About Course': '',
            'Course Syllabus': ''
        }

    all_data.append(course_data)
    time.sleep(1)

# === Save to Excel ===
driver.quit()
df_out = pd.DataFrame(all_data)
output_path = "C:\\Users\\taslim.siddiqui\\Downloads\\eduonix_scraped_output_21.xlsx"
df_out.to_excel(output_path, index=False)
print("✅ All courses saved to:", output_path)


Scraping (1/17): https://www.eduonix.com/ximr-certificate-program-in-business-strategy
Scraping (2/17): https://www.eduonix.com/game-development-edegree
Scraping (3/17): https://www.eduonix.com/fundamentals-of-copyright-fair-use-creative-common-dmca-search-copyright-legally-use-image-video
Scraping (4/17): https://www.eduonix.com/programming-for-the-internet
Scraping (5/17): https://www.eduonix.com/fullstack-javascript-developer-edegree
Scraping (6/17): https://www.eduonix.com/business-analytics-for-beginners
Scraping (7/17): https://www.eduonix.com/cybersecurity-edegree
Scraping (8/17): https://www.eduonix.com/matlab-for-beginners
Scraping (9/17): https://www.eduonix.com/exam-dp-500-for-microsoft-power-bi-amp-azure-services
Scraping (10/17): https://www.eduonix.com/learn-oracle-integration-cloud
Scraping (11/17): https://www.eduonix.com/lifetime-ethical-hacking-and-cyber-security-membership
Scraping (12/17): https://www.eduonix.com/learn-integration-in-oracle-integration-oic-from-scra

Extracted course syllbaus for excel file like https://www.eduonix.com/efficient-kubernetes-management-with-python-and-go

In [7]:
import os
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# -------------------- DRIVER SETUP --------------------
def get_driver(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# === Utility: Check if text is a question ===
def is_question(text):
    question_starters = ['why', 'how', 'what', 'when', 'can', 'do', 'does', 'is', 'are', 'which']
    text = text.strip().lower()
    return text.endswith('?') or any(text.startswith(q) for q in question_starters)

# === Input Excel File Path ===
input_file = r"C:\Users\taslim.siddiqui\Downloads\Eduonix (16).xlsx"  # 👈 Change this as needed
output_file = r"C:\Users\taslim.siddiqui\Downloads\eduonix_syllabus_output.xlsx"

# === Read all course links ===
df_links = pd.read_excel(input_file)
course_links = df_links['Course Link'].dropna().tolist()

# === Setup Headless Chrome ===
driver = get_driver(headless=True)

all_data = []

for i, course_link in enumerate(course_links, start=1):
    print(f"\n🔗 Processing ({i}/{len(course_links)}): {course_link}")
    try:
        driver.get(course_link)
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # === Extract Course Name ===
        course_name_tag = soup.find('h1', class_='productTopHeading')
        course_name = course_name_tag.get_text(strip=True) if course_name_tag else "Unknown Course Name"

        # === Extract Price ===
        price_tag = soup.find('span', id='mnumber')
        price = price_tag.get_text(strip=True) if price_tag else "Unknown Price"

        # === Extract Syllabus ===
        course_titles = []
        for btn in soup.find_all('button', class_='edHead'):
            title_tag = btn.find('span', class_='edCardTitle')
            if title_tag:
                title = title_tag.get_text(strip=True)
                if title:
                    course_titles.append(title)

        syllabus_blocks = soup.find_all(['div', 'span', 'h4'])
        syllabus_content = ""
        current_course_index = -1

        for block in syllabus_blocks:
            if block.name == 'span' and 'edCardTitle' in block.get('class', []):
                current_course_index += 1
                syllabus_content += f"\n🎓 {course_titles[current_course_index]}\n"

            elif block.name == 'h4' and 'curriculumName' in block.get('class', []):
                module_title_elem = block
                module_num_elem = block.find_previous('div', class_='moduleHeading')
                module_title = module_title_elem.get_text(strip=True)
                module_num = module_num_elem.get_text(strip=True) if module_num_elem else ''
                full_module_title = f"{module_num} {module_title}".strip()
                syllabus_content += f"\n🔸 {full_module_title}\n"

            elif block.name == 'span' and 'secInfo' in block.get('class', []):
                b_tag = block.find('b')
                if b_tag:
                    title = b_tag.get_text(strip=True)
                    if 'frequently asked' in title.lower() or 'faq' in title.lower() or is_question(title):
                        continue
                    syllabus_content += f"\n📘 {title.strip()}\n"

            elif block.name == 'div' and 'syllbusMain' in block.get('class', []):
                desc_div = block.find('div', class_='syllbusDesc')
                if desc_div:
                    for btn in desc_div.find_all('button'):
                        btn.decompose()
                    lesson_text = desc_div.get_text(strip=True)
                    if lesson_text:
                        syllabus_content += f"- {lesson_text.strip()}\n"

        if not syllabus_content.strip():
            syllabus_content = "No Syllabus Found"

        all_data.append({
            "Course Name": course_name,
            "Course Link": course_link,
            "Price": price,
            "Syllabus": syllabus_content.strip()
        })

        print(f"✅ Done: {course_name}")

    except Exception as e:
        print(f"❌ Error processing {course_link}: {e}")
        all_data.append({
            "Course Name": "Error",
            "Course Link": course_link,
            "Price": "N/A",
            "Syllabus": str(e)
        })

# === Close Driver ===
driver.quit()

# === Save all results to Excel ===
output_df = pd.DataFrame(all_data)
output_df.to_excel(output_file, index=False)
print(f"\n✅ All data saved to: {output_file}")



🔗 Processing (1/282): https://www.eduonix.com/business-data-analytics-bootcamp-master-ai-powered-insights
✅ Done: Business Data Analytics Bootcamp: Master AI Powered Insights

🔗 Processing (2/282): https://www.eduonix.com/complete-ai-for-professionals
✅ Done: Complete AI for Professionals

🔗 Processing (3/282): https://www.eduonix.com/ai-agents-for-small-business-owners-and-entrepreneurs
✅ Done: AI Agents For Small Business Owners And Entrepreneurs

🔗 Processing (4/282): https://www.eduonix.com/devops-labs-14-real-devops-projects
✅ Done: DevOps Labs: 14 Real DevOps Projects

🔗 Processing (5/282): https://www.eduonix.com/efficient-kubernetes-management-with-python-and-go
✅ Done: Efficient Kubernetes Management with Python and Go

🔗 Processing (6/282): https://www.eduonix.com/mendix-magic-mobilize-your-responsive-apps-with-no-code
✅ Done: Mendix Magic: Mobilize Your Responsive Apps with No Code

🔗 Processing (7/282): https://www.eduonix.com/vibe-coding
✅ Done: Vibe Coding For Beginners


Extract course syllabus and price cours https://www.eduonix.com/c-mastery-from-fundamentals-to-advanced-techniques-bundlee link 

In [17]:
import os
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# -------------------- DRIVER SETUP --------------------
def get_driver(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# === Main function to extract syllabus from a course URL ===
def extract_syllabus_from_url(url, headless=False):
    print(f"🔗 Processing: {url}")
    driver = get_driver(headless=headless)
    
    try:
        print("🌐 Loading page...")
        driver.get(url)
        time.sleep(5)
        
        # Scroll to make sure syllabus section loads
        driver.execute_script("window.scrollTo(0, 1000);")
        time.sleep(2)
        
        # EXPAND ALL COURSES - Multiple strategies
        print("📖 Expanding all courses...")
        
        # Strategy 1: Click all course headers
        try:
            course_headers = driver.find_elements("css selector", ".dealsSyllabusHeading")
            print(f"   Found {len(course_headers)} course headers")
            for i, course_header in enumerate(course_headers):
                print(f"   Expanding course {i+1}...")
                driver.execute_script("arguments[0].click();", course_header)
                time.sleep(3)  # Wait for content to load
        except Exception as e:
            print(f"   ⚠️ Could not expand via headers: {e}")
        
        # Strategy 2: Expand all sections within courses
        try:
            section_buttons = driver.find_elements("css selector", ".edegreeSylBtn, [data-target*='sectionBody']")
            print(f"   Found {len(section_buttons)} section buttons")
            for button in section_buttons:
                try:
                    # Only click if it's collapsed
                    if 'collapsed' in button.get_attribute('class'):
                        driver.execute_script("arguments[0].click();", button)
                        time.sleep(1)
                except:
                    pass
        except Exception as e:
            print(f"   ⚠️ Could not expand sections: {e}")
        
        # Final scroll to ensure everything is loaded
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # === Extract Course Name ===
        course_name_tag = soup.find('h1', class_='productTopHeading')
        course_name = course_name_tag.get_text(strip=True) if course_name_tag else "Unknown Course Name"
        print(f"📚 Course Name: {course_name}")

        # === Extract Price ===
        price_tag = soup.find('span', id='mnumber')
        if not price_tag:
            price_tag = soup.find('span', class_='newPrice')
        price = price_tag.get_text(strip=True) if price_tag else "Unknown Price"
        print(f"💰 Price: {price}")

        # === Extract Syllabus - EXACT FORMAT WITH BULLET POINTS ===
        syllabus_content = ""
        
        # Find the syllabus section
        syllabus_section = soup.find('section', id='syllabus')
        
        if syllabus_section:
            print("✅ Found syllabus section")
            
            # Find all course accordion items
            accordion_links = syllabus_section.find_all('a', class_='accordionDealBody')
            print(f"📖 Found {len(accordion_links)} courses in the bundle")
            
            for i, accordion_link in enumerate(accordion_links, 1):
                # Get course title
                course_title_div = accordion_link.find('div', class_='dealsSyllabusHeading')
                if course_title_div:
                    course_title = course_title_div.get_text(strip=True)
                    # Skip modal titles
                    if 'modal' not in course_title.lower():
                        # Remove "Course X: " prefix if it exists
                        clean_course_title = re.sub(r'^Course\s*\d+\s*:\s*', '', course_title)
                        # Format: "Course X: Clean Title"
                        syllabus_content += f"Course {i}: {clean_course_title}\n\n"
                        print(f"🎓 Processing: {clean_course_title}")
                        
                        # Find the collapse content for this course
                        collapse_href = accordion_link.get('href', '')
                        if collapse_href.startswith('#'):
                            collapse_id = collapse_href[1:]
                            collapse_content = syllabus_section.find('div', id=collapse_id)
                            
                            if collapse_content:
                                # Look for the MainSyllabus div inside collapse content
                                main_syllabus = collapse_content.find('div', class_='MainSyllabus')
                                if main_syllabus:
                                    # Find the accordion inside MainSyllabus
                                    accordion = main_syllabus.find('div', id='accordion')
                                    if accordion:
                                        # Extract all card sections
                                        cards = accordion.find_all('div', class_='card')
                                        print(f"   📋 Found {len(cards)} sections")
                                        
                                        for card_idx, card in enumerate(cards, 1):
                                            # Extract section header - Remove duplicate "Section X: " prefix
                                            section_header = card.find('span', class_='secInfo')
                                            if section_header:
                                                # Get the COMPLETE text including "Preview inside"
                                                section_title = section_header.get_text(strip=True)
                                                # Remove "Section X: " prefix if it exists
                                                clean_section_title = re.sub(r'^Section\s*\d+\s*:\s*', '', section_title)
                                                # Replace &nbsp; with space and format
                                                clean_section_title = clean_section_title.replace('&nbsp;', ' ')
                                                syllabus_content += f"Section {card_idx}: {clean_section_title}\n\n"
                                                print(f"   📋 Section {card_idx}: {clean_section_title}")
                                            
                                            # Find the section body (lectures)
                                            section_body = card.find('div', class_='collapse')
                                            if section_body:
                                                # Extract lectures
                                                lectures = section_body.find_all('div', class_='syllbusMain')
                                                print(f"   📝 Found {len(lectures)} lectures")
                                                
                                                for lecture in lectures:
                                                    seq_num = lecture.find('div', class_='syllbusSwq')
                                                    desc = lecture.find('div', class_='syllbusDesc')
                                                    
                                                    if seq_num and desc:
                                                        # Get the complete lecture text including Preview button text
                                                        lecture_text = desc.get_text(strip=True)
                                                        # Replace &nbsp; with space
                                                        lecture_text = lecture_text.replace('&nbsp;', ' ')
                                                        # Format as bullet point
                                                        syllabus_content += f"-{lecture_text}\n"
                                                
                                                # Add empty line after each section's lectures
                                                syllabus_content += "\n"
                                            else:
                                                syllabus_content += f"-No lectures found in this section\n\n"
                                
                                print(f"   ✅ Course {i} completed")
                                
                                # Add separator between courses
                                if i < len(accordion_links):
                                    syllabus_content += "=" * 80 + "\n\n"
                            else:
                                syllabus_content += f"No content found for this course\n\n"
                                print(f"   ❌ No content found for course {i}")
                        else:
                            syllabus_content += f"Invalid collapse reference\n\n"
                            print(f"   ❌ Invalid collapse reference: {collapse_href}")
        
        else:
            print("❌ No syllabus section found")
            syllabus_content = "No syllabus section found"
        
        return {
            "Course Name": course_name,
            "Course Link": url,
            "Price": price,
            "Syllabus": syllabus_content.strip()
        }

    except Exception as e:
        print(f"❌ Error processing {url}: {e}")
        return {
            "Course Name": "Error",
            "Course Link": url,
            "Price": "N/A",
            "Syllabus": f"Error: {str(e)}"
        }
    
    finally:
        driver.quit()
        print("🚗 Browser closed\n")

# === Process multiple courses from Excel ===
def process_courses_from_excel(input_file, output_file, headless=True):
    """
    Process all course links from an Excel file and extract syllabus data
    """
    # Read course links from Excel
    df_links = pd.read_excel(input_file)
    course_links = df_links['Course Link'].dropna().tolist()
    
    print(f"📊 Found {len(course_links)} courses to process")
    
    all_data = []
    
    for i, course_link in enumerate(course_links, 1):
        print(f"\n{'='*60}")
        print(f"Processing {i}/{len(course_links)}")
        print(f"{'='*60}")
        
        # Extract syllabus data
        course_data = extract_syllabus_from_url(course_link, headless=headless)
        all_data.append(course_data)
        
        # Save progress after each course
        temp_df = pd.DataFrame(all_data)
        temp_output = output_file.replace('.xlsx', '_temp.xlsx')
        temp_df.to_excel(temp_output, index=False)
        print(f"💾 Progress saved to: {temp_output}")
    
    # Save final results
    final_df = pd.DataFrame(all_data)
    final_df.to_excel(output_file, index=False)
    print(f"\n✅ All data saved to: {output_file}")
    
    return final_df

# === Test with single course ===
def test_single_course():
    test_link = "https://www.eduonix.com/c-mastery-from-fundamentals-to-advanced-techniques-bundle"
    output_file = r"C:\Users\taslim.siddiqui\Downloads\eduonix_c_mastery_test.xlsx"
    
    print("🧪 SINGLE COURSE TEST MODE")
    result = extract_syllabus_from_url(test_link, headless=False)
    
    # Save result
    df = pd.DataFrame([result])
    df.to_excel(output_file, index=False)
    print(f"✅ Test results saved to: {output_file}")
    
    # Print extracted content
    print("\n" + "="*80)
    print("EXTRACTED SYLLABUS CONTENT:")
    print("="*80)
    print(result['Syllabus'])
    
    return result

# === Main execution ===
if __name__ == "__main__":
    # CHOOSE ONE OF THE FOLLOWING OPTIONS:
    
    # Option 1: Test with single course
    test_single_course()
    
    # Option 2: Process all courses from Excel (uncomment to use)
    
    input_file = r"C:\Users\taslim.siddiqui\Downloads\course link pending eduonix.xlsx"
    output_file = r"C:\Users\taslim.siddiqui\Downloads\eduonix_syllabus_complete.xlsx"
    process_courses_from_excel(input_file, output_file, headless=True)
    

🧪 SINGLE COURSE TEST MODE
🔗 Processing: https://www.eduonix.com/c-mastery-from-fundamentals-to-advanced-techniques-bundle
🌐 Loading page...
📖 Expanding all courses...
   Found 3 course headers
   Expanding course 1...
   Expanding course 2...
   Expanding course 3...
   Found 13 section buttons
📚 Course Name: C++ Mastery: From Fundamentals to Advanced Techniques Bundle
💰 Price: ₹2,000.00
✅ Found syllabus section
📖 Found 3 courses in the bundle
🎓 Processing: Learn C++ Fundamentals : Coding for Absolute Beginners
   📋 Found 10 sections
   📋 Section 1: IntroductionPreview inside
   📝 Found 5 lectures
   📋 Section 2: Setting up Local DevelopmentPreview inside
   📝 Found 8 lectures
   📋 Section 3: C++ BasicsPreview inside
   📝 Found 11 lectures
   📋 Section 4: C++ Data typePreview inside
   📝 Found 6 lectures
   📋 Section 5: C++ Operators
   📝 Found 5 lectures
   📋 Section 6: C++ Strings
   📝 Found 8 lectures
   📋 Section 7: C++ Decision Making
   📝 Found 6 lectures
   📋 Section 8: C++ Cont